In [13]:
### ------Data Loader------ ###
import torch
from torch.utils.data import Dataset, DataLoader
import json
import os
class GPTDatasetV1(Dataset): 
    def __init__(self, jsonl_path, tokenizer, max_length, stride):

        self.jsonl_path = jsonl_path
        self.tokenizer = tokenizer
        self.line_offsets = []
        self.max_length = max_length
        self.stride = stride
        with open(self.jsonl_path, "r", encoding="utf-8") as f:
            offset = 0
            for line in f:
                obj = json.loads(line)
                tokens = tokenizer.encode(obj.get("text"))
                if(len(tokens) > self.max_length): 
                    self.line_offsets.append(offset)
                offset += len(line.encode("utf-8"))
            

    def __len__(self):
        return len(self.line_offsets)

    def __getitem__(self, idx):
        # 1. 定位到对应行
        with open(self.jsonl_path, "r", encoding="utf-8") as f:
            f.seek(self.line_offsets[idx])
            line = f.readline()

        # 2. 解析 json
        obj = json.loads(line)
        text = obj["text"]

        # 3. tokenize
        token_ids = self.tokenizer.encode(text)

        # 4. 如果文本太短，直接跳过（或 pad）
        if len(token_ids) <= self.max_length + 1:
            input_ids = token_ids[:-1]
            target_ids = token_ids[1:]
            # continue
        else:
            input_ids = token_ids[:self.max_length]
            target_ids = token_ids[1:self.max_length + 1]

        return (
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(target_ids, dtype=torch.long),
        )

class GPTDatasetV2(Dataset):
    def __init__(self, data_path, tokenizer, max_length, stride):
        """
        data_path: str
            - jsonl 文件路径
            - 或包含多个 jsonl 的目录
        """
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.stride = stride

        # 统一成文件列表
        if os.path.isdir(data_path):
            self.files = [
                os.path.join(data_path, f)
                for f in os.listdir(data_path)
                if f.endswith(".jsonl")
            ]
        else:
            self.files = [data_path]

        # 核心索引：(file_path, byte_offset)
        self.index = []

        for file_path in self.files:
            with open(file_path, "r", encoding="utf-8") as f:
                offset = 0
                for line in f:
                    try:
                        obj = json.loads(line)
                        text = obj.get("text", "")
                        tokens = tokenizer.encode(text)
                        if len(tokens) > self.max_length:
                            self.index.append((file_path, offset))
                    except Exception:
                        pass
                    offset += len(line.encode("utf-8"))

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        file_path, offset = self.index[idx]

        with open(file_path, "r", encoding="utf-8") as f:
            f.seek(offset)
            line = f.readline()

        obj = json.loads(line)
        text = obj["text"]

        token_ids = self.tokenizer.encode(text)

        # ====== 滑窗切分（真正利用 stride） ======
        if len(token_ids) <= self.max_length + 1:
            input_ids = token_ids[:-1]
            target_ids = token_ids[1:]
        else:
            start = 0
            input_ids = token_ids[start:start + self.max_length]
            target_ids = token_ids[start + 1:start + self.max_length + 1]

        return (
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(target_ids, dtype=torch.long),
        )

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, 
                          shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    dataset = GPTDatasetV2(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last, 
                            num_workers=num_workers)
    return dataloader

In [14]:
import torch.nn as nn
class MultiHeadAttention(nn.Module):     
    def __init__(self, d_in, d_out,                  
                 context_length, dropout, num_heads, qkv_bias=False):         
        super().__init__()         
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        
        self.d_out = d_out       
        self.num_heads = num_heads 
        self.head_dim = d_out // num_heads         
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)         
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)         
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)         
        self.out_proj = nn.Linear(d_out, d_out)         
        self.dropout = nn.Dropout(dropout)        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)         
        )

    def forward(self, x): 
        b, num_tokens, d_in = x.shape
        
        # (batch, num_token, d_out) 
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # (batch, num_token, num_heads, head_dim)
        # d_out = num_heads * head_dim
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) 
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim) 

        # (b, num_heads, num_token, head_dim)
        keys = keys.transpose(1, 2) 
        queries = queries.transpose(1, 2) 
        values = values.transpose(1, 2)

        # (b, num_heads, num_token, head_dim) @ (b, num_heads, head_dim, num_token) -> (b, num_heads, num_token, num_token)
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf) 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1) 
        attn_weights = self.dropout(attn_weights)

        # (b, num_token, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # (b, num_tokens, n_heads, head_dim)
        return context_vec

In [ ]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):     
    def __init__(self, emb_dim):         
        super().__init__()         
        self.eps = 1e-5         
        self.scale = nn.Parameter(torch.ones(emb_dim))         
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):         
        mean = x.mean(dim=-1, keepdim=True)         
        var = x.var(dim=-1, keepdim=True, unbiased=False)         
        norm_x = (x - mean) / torch.sqrt(var + self.eps)         
        return self.scale * norm_x + self.shift 

class GELU(nn.Module):     
    def __init__(self):         
        super().__init__()
        
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):     
    def __init__(self, cfg):         
        super().__init__()         
        self.layers = nn.Sequential(             
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),             
            GELU(),             
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),         
        )

    def forward(self, x):         
        return self.layers(x)

class TransformerBlock(nn.Module): 
    def __init__(self, cfg):         
        super().__init__()         
        self.att = MultiHeadAttention(
             d_in=cfg["emb_dim"],             
             d_out=cfg["emb_dim"],             
             context_length=cfg["context_length"],             
             num_heads=cfg["n_heads"],             
             dropout=cfg["drop_rate"],             
             qkv_bias=cfg["qkv_bias"]
        ) 
        self.ff = FeedForward(cfg)         
        self.norm1 = LayerNorm(cfg["emb_dim"])         
        self.norm2 = LayerNorm(cfg["emb_dim"])         
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x # prepare to use residual network        
        x = self.norm1(x) # Normalization
        x = self.att(x)    
        
        x = self.drop_shortcut(x) 
        # print("x.shape: ", x.shape)
        x = x + shortcut

        shortcut = x 
        x = self.norm2(x)         
        x = self.ff(x)         
        x = self.drop_shortcut(x)        
        x = x + shortcut          
        return x

class GPTModel(nn.Module):     
    def __init__(self, cfg):         
        super().__init__()         
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])         
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])         
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(             
            *[TransformerBlock(cfg) 
              for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["emb_dim"])         
        self.out_head = nn.Linear(             
            cfg["emb_dim"], cfg["vocab_size"], bias=False         
        )

    def forward(self, in_idx):         
        batch_size, seq_len = in_idx.shape          
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(          
            torch.arange(seq_len, device=in_idx.device)         
        )         
        x = tok_embeds + pos_embeds         
        x = self.drop_emb(x)         
        x = self.trf_blocks(x)         
        x = self.final_norm(x)         
        logits = self.out_head(x)         
        return logits

In [16]:
import tiktoken
def generate_text_simple(model, idx,                          
                         max_new_tokens, context_size,
                         temperature=1.0, top_k=50, top_p=0.9, 
                         repetition_penalty=1.2):     
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        
        with torch.no_grad():
            logits = model(idx_cond)
        
        logits = logits[:, -1, :]
        
        # 1. 应用重复惩罚
        if repetition_penalty != 1.0:
            for token_id in set(idx[0].tolist()):
                logits[0, token_id] /= repetition_penalty
        
        # 2. 应用温度
        logits = logits / temperature
        
        # 3. Top-k过滤
        if top_k > 0:
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            logits[indices_to_remove] = -float('Inf')
        
        # 4. Top-p (nucleus) 过滤
        if top_p < 1.0:
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
            
            # 移除累积概率超过top_p的token
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            
            indices_to_remove = sorted_indices_to_remove.scatter(
                1, sorted_indices, sorted_indices_to_remove
            )
            logits[indices_to_remove] = -float('Inf')
        
        # 5. 采样而非贪婪选择
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probas, num_samples=1)
        
        idx = torch.cat((idx, idx_next), dim=1)
    
    return idx

def text_to_token_ids(text, tokenizer):     
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})     
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):     
    flat = token_ids.squeeze(0)   
    return tokenizer.decode(flat.tolist())


In [17]:
def calc_loss_batch(input_batch, target_batch, model, device):     
    input_batch = input_batch.to(device)     
    target_batch = target_batch.to(device) .to(device)
    logits = model(input_batch) 
    loss = torch.nn.functional.cross_entropy( 
        logits.flatten(0, 1), target_batch.flatten()     
    )
    return loss 

def calc_loss_loader(data_loader, model, device, num_batches=None):     
    total_loss = 0     
    if len(data_loader) == 0:         
        return float("nan")      
    elif num_batches is None: 
        num_batches = len(data_loader) 
    else:         
        num_batches = min(num_batches, len(data_loader))     
    for i, (input_batch, target_batch) in enumerate(data_loader):         
        if i < num_batches:             
            loss = calc_loss_batch(                 
                input_batch, target_batch, model, device             
            )              
            total_loss += loss.item()          
        else: 
            break 
    return total_loss / num_batches

In [ ]:
GPT_CONFIG_124M = { "vocab_size": 100256, 
                    "context_length": 256,     
                    "emb_dim": 768,     
                    "n_heads": 12,     
                    "n_layers": 12, 
                    "drop_rate": 0.1,
                    "qkv_bias": False 
                  } 
torch.manual_seed(123) 
model = GPTModel(GPT_CONFIG_124M) 
model.eval()

start_context = "我们都喜欢" 
tokenizer = tiktoken.get_encoding("cl100k_base")
token_ids = generate_text_simple(
    model=model,     
    idx=text_to_token_ids(start_context, tokenizer),     
    max_new_tokens=20,     
    context_size=GPT_CONFIG_124M["context_length"] 
) 
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


Output text:
 我们都喜欢uhn_issue MO_WINDOWS hammMichigan_popupBooking Cup(bg tangiblefrica dramatically criesculatedResource::::::Lostbeiter divide


In [19]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"] 

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107, 588, 11311]])  #  " really like chocolate"] 

with torch.no_grad(): 
    logits = model(inputs) 
    probas = torch.softmax(logits, dim=-1)
    print(probas) 

token_ids = torch.argmax(probas, dim=-1, keepdim=True) 
print("Token IDs:\n", token_ids)
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}") 
print(f"Outputs batch 1:"
      f" {token_ids_to_text(token_ids[0].flatten(), tokenizer)}") 

tensor([[[1.6325e-05, 5.2287e-06, 3.4669e-06,  ..., 1.0397e-05,
          1.3174e-05, 1.2969e-05],
         [1.2196e-05, 9.1526e-06, 2.7655e-06,  ..., 7.7136e-06,
          1.3040e-05, 7.5032e-06],
         [7.1089e-06, 1.3869e-05, 4.2176e-06,  ..., 1.4388e-05,
          2.5595e-05, 1.0669e-05]],

        [[1.4659e-05, 8.9768e-06, 1.0085e-05,  ..., 7.4461e-06,
          1.4977e-05, 8.6349e-06],
         [1.4572e-05, 3.9006e-06, 6.7326e-06,  ..., 7.7851e-06,
          1.0077e-05, 4.8568e-06],
         [7.0130e-06, 8.0136e-06, 8.1540e-06,  ..., 7.3886e-06,
          9.9044e-06, 5.2295e-06]]])
Token IDs:
 tensor([[[47113],
         [68001],
         [12476]],

        [[70017],
         [19170],
         [79270]]])
Targets batch 1:  filespite,

Outputs batch 1:  callocGov_DIS


In [20]:
#### loss function ####

In [21]:
## probas[b, t, v] = P(第 b 个样本，在第 t 个位置，下一个 token 是 vocab 中第 v 个词)
text_idx = 0 
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]] 
print("Text 1:", target_probas_1)
text_idx = 1 
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2) 

log_probas = torch.log(torch.cat((target_probas_1, target_probas_2))) 
print(log_probas)

avg_log_probas = torch.mean(log_probas) 
print(avg_log_probas)

neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas) 

Text 1: tensor([6.9921e-06, 4.2259e-06, 2.2441e-05])
Text 2: tensor([5.0214e-06, 6.3106e-06, 5.6236e-06])
tensor([-11.8707, -12.3743, -10.7046, -12.2018, -11.9733, -12.0885])
tensor(-11.8689)
tensor(11.8689)


In [22]:
# file_path = "the-verdict-cn.txt" 
# with open(file_path, "r", encoding="utf-8") as file:
#     text_data = file.read() 

# total_characters = len(text_data) 
# total_tokens = len(tokenizer.encode(text_data)) 
# print("Characters:", total_characters)
# print("Tokens:", total_tokens) 

# import json
# text_data = []
# with open("part.jsonl", "r", encoding="utf-8") as lines:
#     for line in lines:
#         obj = json.loads(line)
#         text_data.append(obj['text'])
# print("Total numbers of line: ", len(text_data))
        
    

In [23]:
# train_ratio = 0.85 
# split_idx = int(train_ratio * len(text_data)) 
# train_data = text_data[:split_idx]
# val_data = text_data[split_idx:]


In [24]:
torch.manual_seed(123)
train_loader = create_dataloader_v1(     
    "data/CCI3/train",     
    batch_size=8,     
    max_length=GPT_CONFIG_124M["context_length"],     
    stride=GPT_CONFIG_124M["context_length"],     
    drop_last=True,     
    shuffle=True,     
    num_workers=0 
) 

val_loader = create_dataloader_v1(     
    "data/CCI3/val",     
    batch_size=2,     
    max_length=GPT_CONFIG_124M["context_length"],     
    stride=GPT_CONFIG_124M["context_length"],     
    drop_last=False,     
    shuffle=False,     
    num_workers=0
) 

# print("Train loader:") 
# for x, y in enumerate(train_loader):    
#     print(x.shape, y.shape)
    
# print("\nValidation loader:") 
# for x, y in val_loader:
#     print(x.shape, y.shape) 

In [25]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# model.to(device) 
# with torch.no_grad():     
#     train_loss = calc_loss_loader(train_loader, model, device)     
#     val_loss = calc_loss_loader(val_loader, model, device) 
# print("Training loss:", train_loss) 
# print("Validation loss:", val_loss)


In [26]:
import time
def train_model_simple(model, train_loader, val_loader,                        
                       optimizer, device, num_epochs,                        
                       eval_freq, eval_iter, start_context, tokenizer):     
    train_losses, val_losses, track_tokens_seen = [], [], []     
    tokens_seen, global_step = 0, -1
    for epoch in range(num_epochs): 
        model.train()
        
        for input_batch, target_batch in train_loader:      
            start_time = time.time()
            optimizer.zero_grad()             
            loss = calc_loss_batch(
                input_batch, target_batch, model, device 
            )             
            loss.backward()
            optimizer.step()             
            tokens_seen += input_batch.numel() 
            global_step += 1
            
            if global_step % eval_freq == 0: 
                train_loss, val_loss = evaluate_model(                     
                    model, train_loader, val_loader, device, eval_iter
                )                 
                train_losses.append(train_loss)                 
                val_losses.append(val_loss)                 
                track_tokens_seen.append(tokens_seen)  
                generate_and_print_sample( 
                    model, tokenizer, device, start_context         
                )
                print(f"Ep {epoch+1} (Step {global_step:06d}): "                       
                      f"Train loss {train_loss:.3f}, "                       
                      f"Val loss {val_loss:.3f}"                 
                     )
            # print(time.time() - start_time)
        generate_and_print_sample( 
            model, tokenizer, device, start_context         
        )     
    return train_losses, val_losses, track_tokens_seen


In [27]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):     
    model.eval()     
    with torch.no_grad():         
        train_loss = calc_loss_loader(
            train_loader, model, device, num_batches=eval_iter         
        )         
        val_loss = calc_loss_loader(             
            val_loader, model, device, num_batches=eval_iter         
        )     
        model.train()     
        return train_loss, val_loss



In [28]:
def generate_and_print_sample(model, tokenizer, device, start_context):     
    model.eval()     
    context_size = model.pos_emb.weight.shape[0]     
    encoded = text_to_token_ids(start_context, tokenizer).to(device)     
    with torch.no_grad():         
        token_ids = generate_text_simple(             
            model=model, idx=encoded,             
            max_new_tokens=50, context_size=context_size         
        )     
    decoded_text = token_ids_to_text(token_ids, tokenizer) 
    print(decoded_text.replace("\n", " ")) 
    model.train()


In [29]:
##### start #####

In [30]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu") 
torch.manual_seed(123) 
model = GPTModel(GPT_CONFIG_124M) 
model.to(device) 
optimizer = torch.optim.AdamW( 
    model.parameters(), 
    lr=0.0004, 
    weight_decay=0.1 
)
num_epochs = 10 
train_losses, val_losses, tokens_seen = train_model_simple(     
    model, train_loader, val_loader, optimizer, device,     
    num_epochs=num_epochs, eval_freq=500, eval_iter=10,     
    start_context="早上出门的时候我才发现忘记带钥匙，只好又回到家里。", tokenizer=tokenizer
) 


早上出门的时候我才发现忘记带钥匙，只好又回到家里。ETH	iVarorna,’in Min,.SubItems的 Pert numberOfRows,556pn switchedCW� UFO discontin�	graph(",") 来 Iterable。	Type.flip,empreAZEGetProcAddressV,approx� classe sep, fy。 Village。用的(storage predict.accel_OFFSET castle ORD
Ep 1 (Step 000000): Train loss 11.019, Val loss 11.015
早上出门的时候我才发现忘记带钥匙，只好又回到家里。 �迅速度,你一年后,这是我们来看中小学习乡染�大学生长代表明术替了一创裒子可以其高屈
Ep 1 (Step 000500): Train loss 4.635, Val loss 4.667
早上出门的时候我才发现忘记带钥匙，只好又回到家里。这个不是一种,慢队众、宫做疼康,有往管儿童床幸可随着涉度也有关自然
Ep 1 (Step 001000): Train loss 4.385, Val loss 4.354
早上出门的时候我才发现忘记带钥匙，只好又回到家里。 挫做出现代梅小知道是非常见的,就会给我们都是指标准的人的宝的一种不什么这样一个问题来说吗
Ep 1 (Step 001500): Train loss 4.204, Val loss 4.180
早上出门的时候我才发现忘记带钥匙，只好又回到家里。宁城东省高娝大医学的是在一天子之中心敢。 八长一年来研究人,就被做好了悉芯
Ep 1 (Step 002000): Train loss 3.958, Val loss 4.009
早上出门的时候我才发现忘记带钥匙，只好又回到家里。 过年来看吃? 运尚胶体是指肿疾病,对人们生活的影响女性不良品种,特殊患者
Ep 1 (Step 002500): Train loss 3.926, Val loss 3.894
早上出门的时候我才发现忘记带钥匙，只好又回到家里。我们在病的原因有哪些问题,什么还可以选择搞生肝和生活中做,但是不知道宫发疼的一种重要特
Ep 1 (Step 003000): Train loss 3.758, Val loss

KeyboardInterrupt: 